# Pasos:

1. Particionar datos de Entrenamiento (80%) y Pruebas(20%) 
2. Entrenar Modelos a evaluar:
    - XGBoost
    - SVM
    - Ensamble Modelo (RandomForest o Arbol de decisión)
    - Naive Bayes - Gausiano
    - Redes Neuronales
3. Evaluar los hiperparametros (RandomSearchCV)
4. Entrenar modelos con mejores hiperparámetros encontrados en 3.
5. Evaluar modelos con data de pruebas (Accuracy)

## 1) Lectura de datos - Distribución de conjunto de datos

La distribución del conjunto de datos será de la siguiente manera: 

- Entrenamiento (80%)
- Pruebas (20%)

--- Ya fue aplicado en 2. Pre-procesamiento de datos

In [1]:
import pandas as pd
import numpy as np
import os
import csv
import matplotlib.pyplot as plt

from sklearn import preprocessing, neighbors
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [2]:
#Lectura del dataset


#Lectura del dataset

X_TRAIN_DF = pd.read_csv('data/03_entrada_modelo/X_MI_entrenamiento_caracterizado.csv', header=0, sep=",")
X_TEST_DF  = pd.read_csv('data/03_entrada_modelo/X_MI_pruebas_caracterizado.csv', header=0, sep=",")
Y_TRAIN_DF = pd.read_csv('data/03_entrada_modelo/y_MI_entrenamiento_caracterizado.csv', header=0, sep=",")
Y_TEST_DF  = pd.read_csv('data/03_entrada_modelo/y_MI_pruebas_caracterizado.csv', header=0, sep=",")

In [3]:
var_objetivo_lst = [
    'FIBR_PREDS','PREDS_TAH','JELUD_TAH','FIBR_JELUD',
    'A_V_BLOK','OTEK_LANC','RAZRIV','DRESSLER','ZSN',
    'REC_IM','P_IM_STEN'
]

## 2) Entrenar Modelos a evaluar

In [4]:
## Resultados con la métrica de exactitud (Accuracy) de cada uno de los modelos.
resultados_modelo_metricas_df = pd.DataFrame(columns=['Variable Objetivo','Modelo','Accuracy Train','Accuracy Test', 'Especificidad', "F1_score"])

### 2.1) Random Forest

Entrenamos el modelo de Random Forest de tipo clasificación.

In [5]:
for var_objetivo in var_objetivo_lst:
    
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    X_test.drop(["var_objetivo"], axis=1, inplace=True)
    
    
    rf = RandomForestClassifier(oob_score = True)
    rf.fit(X_train, y_train)

    print('Exactitud del modelo inicial en entrenamiento:', rf.score(X_train, y_train))
    print('Exactitud del modelo inicial en entrenamiento (Out of Bag):', rf.oob_score_)
    print('Exactitud del modelo inicial en validación:', rf.score(X_test, y_test))

    #### Búsqueda aleatoria de mejores hiperparámetros
    # Definición de Grilla
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_features = ['auto', 10, 11, 12]  # 'auto' equivale a 'sqrt'; None equivale a todas
    max_depth = [int(x) for x in np.linspace(10, 220, num = 11)] + [None]
    min_samples_leaf = [1, 2, 4]

    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_leaf': min_samples_leaf}

    print('Los valores a probar en la búsqueda aleatoria son:')
    pprint(random_grid)

    print()
    print('Si se probara todas las combinaciones se requeriría entrenar', 
          len(random_grid['n_estimators']) *
          len(random_grid['max_features']) *
          len(random_grid['max_depth']) *
          len(random_grid['min_samples_leaf']),
          'modelos'
          )

    rf = RandomForestClassifier(oob_score=True)
    rf_random = RandomizedSearchCV(estimator = rf, 
                                   param_distributions = random_grid, 
                                   cv = 3,          # Validación cruzada 3-fold
                                   verbose=2, 
                                   random_state=0, 
                                   n_jobs = -1      # Paralelizar en todos los cores disponibles
                                   )
    rf_random.fit(X_train, y_train)

    ## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
    rf_random_best = rf_random.best_estimator_

    print('Los hiperparámetros del mejor modelo son:')
    pprint(rf_random.best_params_)
    print()
    
    
    y_pred = rf_random_best.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    f1_score_metric = f1_score(y_test, y_pred, average='weighted')
    
    resultados_modelo_metricas_df=resultados_modelo_metricas_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'Modelo' : 'RandomForest' , 
            'Accuracy Train' : rf_random_best.score(X_train, y_train),
            'Accuracy Test' : rf_random_best.score(X_test, y_test),
            'Especificidad' : specificity,
            'F1_score': f1_score_metric
        },
        ignore_index=True
    )
    
    print('*'*100)
    print(f'{var_objetivo}')
    print('-'*100)
    print('Exactitud luego de búsqueda aleatoria en entrenamiento:', rf_random_best.score(X_train, y_train))
    print('Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag):', rf_random_best.oob_score_)
    print('Exactitud luego de búsqueda aleatoria en validación:', rf_random_best.score(X_test, y_test))
    print('.'*50)
    print('Especificidad luego de búsqueda aleatoria en validación:', specificity)
    print('F1_score luego de búsqueda aleatoria en validación:', f1_score_metric)
    print('*'*100)

Exactitud del modelo inicial en entrenamiento: 1.0
Exactitud del modelo inicial en entrenamiento (Out of Bag): 0.9635067114093959
Exactitud del modelo inicial en validación: 0.9
Los valores a probar en la búsqueda aleatoria son:
{'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 199, 220, None],
 'max_features': ['auto', 10, 11, 12],
 'min_samples_leaf': [1, 2, 4],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

Si se probara todas las combinaciones se requeriría entrenar 1440 modelos
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Los hiperparámetros del mejor modelo son:
{'max_depth': 52,
 'max_features': 10,
 'min_samples_leaf': 1,
 'n_estimators': 1600}

****************************************************************************************************
FIBR_PREDS
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en entrenamiento: 1.0
Exactitud luego

D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Los hiperparámetros del mejor modelo son:
{'max_depth': 220,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'n_estimators': 1400}

****************************************************************************************************
FIBR_JELUD
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en entrenamiento: 0.9972527472527473
Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag): 0.9846938775510204
Exactitud luego de búsqueda aleatoria en validación: 0.9441176470588235
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9968944099378882
F1_score luego de búsqueda aleatoria en validación: 0.919836255228264
****************************************************************************************************
Exactitud del modelo inicial en entrenamiento: 1.0
Exactitud del modelo inicial en entrenamiento (Out of Bag): 0.9886807

D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Los hiperparámetros del mejor modelo son:
{'max_depth': 220,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'n_estimators': 1400}

****************************************************************************************************
RAZRIV
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en entrenamiento: 0.999609984399376
Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag): 0.9890795631825273
Exactitud luego de búsqueda aleatoria en validación: 0.9705882352941176
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9969788519637462
F1_score luego de búsqueda aleatoria en validación: 0.9589991220368743
****************************************************************************************************
Exactitud del modelo inicial en entrenamiento: 1.0
Exactitud del modelo inicial en entrenamiento (Out of Bag): 0.98304416403

### 2.2) Ensamble - XGBoost

Entrenamos el modelo de XGBoost de tipo clasificación.

In [6]:
from xgboost import XGBClassifier
from pprint import pprint
from sklearn.model_selection import GridSearchCV
import time

## XGBoostRegressor
modelXGBC = XGBClassifier()


## Creamos la Grilla
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]
learning_rate = [0.01,0.1]
max_depth = [i for i in range(2,8,2)]
start = time.time()

random_grid = {'max_depth': max_depth,
               'n_estimators': n_estimators,
               'colsample_bytree': [0.2, 0.6, 0.8],
               'min_child_weight': [3, 5, 7],
               'gamma': [0.3, 0.5, 0.7],
               'subsample': [0.4, 0.6, 0.8, 1],
               'learning_rate': learning_rate}

#### Búsqueda aleatoria de mejores hiperparámetros

In [7]:
for var_objetivo in var_objetivo_lst:    
    
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    X_test.drop(["var_objetivo"], axis=1, inplace=True)
    
    xgb_random = RandomizedSearchCV(estimator = modelXGBC, 
                                   param_distributions = random_grid, 
                                   cv = 3,          # Validación cruzada 3-fold
                                   verbose=2, 
                                   random_state=0, 
                                   n_jobs = -1      # Paralelizar en todos los cores disponibles
                                   )
    xgb_random.fit(X_train, y_train)
    ## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
    xgb_best_model = xgb_random.best_estimator_

    print('Los hiperparámetros del mejor modelo son:')
    pprint(xgb_random.best_params_)
    print()


    y_pred = xgb_best_model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    f1_score_metric = f1_score(y_test, y_pred, average='weighted')
    
    score_XGBR_train = xgb_best_model.score(X_train, y_train)
    score_XGBR_val = xgb_best_model.score(X_test, y_test)
    
    resultados_modelo_metricas_df=resultados_modelo_metricas_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'Modelo' : 'XGBoost' , 
            'Accuracy Train' : score_XGBR_train,
            'Accuracy Test' : score_XGBR_val,
            'Especificidad' : specificity,
            'F1_score': f1_score_metric
        },
        ignore_index=True
    )

    print('*'*100)
    print(f'{var_objetivo}')
    print('-'*100)
    print(f"Exactitud luego de búsqueda en grilla en entrenamiento: {score_XGBR_train*100:.3f}%")
    print(f"Exactitud luego de búsqueda en grilla en validación: {score_XGBR_val*100:.3f}%")
    print('.'*50)
    print('Especificidad luego de búsqueda aleatoria en validación:', specificity)
    print('F1_score luego de búsqueda aleatoria en validación:', f1_score_metric)
    print('*'*100)
    end = time.time()
    print(f"{end-start} segundos")

Fitting 3 folds for each of 10 candidates, totalling 30 fits


D:\Users\coliverac\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:00:19] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.6,
 'gamma': 0.5,
 'learning_rate': 0.1,
 'max_depth': 6,
 'min_child_weight': 5,
 'n_estimators': 500,
 'subsample': 0.8}

****************************************************************************************************
FIBR_PREDS
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda en grilla en entrenamiento: 99.958%
Exactitud luego de búsqueda en grilla en validación: 87.941%
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9674267100977199
F1_score luego de búsqueda aleatoria en validación: 0.8532684884977615
***********************

D:\Users\coliverac\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:00:34] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.2,
 'gamma': 0.7,
 'learning_rate': 0.1,
 'max_depth': 6,
 'min_child_weight': 3,
 'n_estimators': 400,
 'subsample': 0.6}

****************************************************************************************************
PREDS_TAH
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda en grilla en entrenamiento: 99.733%
Exactitud luego de búsqueda en grilla en validación: 98.529%
..................................................
Especificidad luego de búsqueda aleatoria en validación: 1.0
F1_score luego de búsqueda aleatoria en validación: 0.9779956427015251
***************************************

...

[13:04:50] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.2,
 'gamma': 0.7,
 'learning_rate': 0.01,
 'max_depth': 4,
 'min_child_weight': 7,
 'n_estimators': 400,
 'subsample': 1}

****************************************************************************************************
ZSN
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda en grilla en entrenamiento: 88.358%
Exactitud luego de búsqueda en grilla en validación: 78.235%
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9840637450199203
F1_score luego de búsqueda aleatoria en validación: 0.7308697774884603
*******************************

D:\Users\coliverac\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:05:05] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.2,
 'gamma': 0.7,
 'learning_rate': 0.1,
 'max_depth': 6,
 'min_child_weight': 3,
 'n_estimators': 400,
 'subsample': 0.6}

****************************************************************************************************
REC_IM
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda en grilla en entrenamiento: 99.246%
Exactitud luego de búsqueda en grilla en validación: 91.176%
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9778481012658228
F1_score luego de búsqueda aleatoria en validación: 0.8907952069716778
***************************

D:\Users\coliverac\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:05:22] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.6,
 'gamma': 0.5,
 'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_weight': 7,
 'n_estimators': 300,
 'subsample': 1}

****************************************************************************************************
P_IM_STEN
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda en grilla en entrenamiento: 98.717%
Exactitud luego de búsqueda en grilla en validación: 90.588%
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9839743589743589
F1_score luego de búsqueda aleatoria en validación: 0.8770351484246947
**************************

### 2.3) SVM

Entrenamos el modelo de SVM de tipo clasificación.

In [8]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

modelSVM = svm.SVC()
# Entrenamiento del modelo base - SVM

#### Búsqueda aleatoria de mejores hiperparámetros

In [9]:
for var_objetivo in var_objetivo_lst:
    # Lectura
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    X_test.drop(["var_objetivo"], axis=1, inplace=True)
    
    # Definición de grilla
    random_grid = {'kernel': ['rbf'],
                   'C': np.logspace(-4,4,9), # [0.0001, 0.001, ..., 10000]
                   'gamma': np.logspace(-4,4,9)  # [0.0001, 0.001, ..., 10000]
                  }

    modelSVM_random = RandomizedSearchCV(estimator = modelSVM, 
                                   param_distributions = random_grid, 
                                   cv = 12,          # Validación cruzada 3-fold
                                   verbose=2, 
                                   random_state=0, 
                                   n_jobs = -1      # Paralelizar en todos los cores disponibles
                                   )
    modelSVM_random.fit(X_train, y_train)

    ## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
    modelSVM_best_model = modelSVM_random.best_estimator_

    print('Los hiperparámetros del mejor modelo son:')
    pprint(modelSVM_random.best_params_)
    print()


    
    
    y_pred = modelSVM_best_model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    f1_score_metric = f1_score(y_test, y_pred, average='weighted')
    
    score_SVM_train = modelSVM_best_model.score(X_train, y_train)
    score_SVM_test = modelSVM_best_model.score(X_test, y_test)
    
    resultados_modelo_metricas_df=resultados_modelo_metricas_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'Modelo' : 'SVM' , 
            'Accuracy Train' : score_SVM_train,
            'Accuracy Test' : score_SVM_test,
            'Especificidad' : specificity,
            'F1_score': f1_score_metric
        },
        ignore_index=True
    )
    
    
    print('*'*100)
    print(f'{var_objetivo}')
    print('-'*100)
    print(f"Exactitud luego de búsqueda aleatoria en grilla en entrenamiento: {score_SVM_train:.3f}")
    print(f"Exactitud luego de búsqueda aleatoria en grilla en validación: {score_SVM_test:.3f}")
    print('.'*50)
    print('Especificidad luego de búsqueda aleatoria en validación:', specificity)
    print('F1_score luego de búsqueda aleatoria en validación:', f1_score_metric)
    print('*'*100)
    end = time.time()
    print(end-start)

Fitting 12 folds for each of 10 candidates, totalling 120 fits
Los hiperparámetros del mejor modelo son:
{'C': 10000.0, 'gamma': 0.01, 'kernel': 'rbf'}

****************************************************************************************************
FIBR_PREDS
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en grilla en entrenamiento: 1.000
Exactitud luego de búsqueda aleatoria en grilla en validación: 0.812
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.8827361563517915
F1_score luego de búsqueda aleatoria en validación: 0.8206971094565735
****************************************************************************************************
354.9387114048004
Fitting 12 folds for each of 10 candidates, totalling 120 fits
Los hiperparámetros del mejor modelo son:
{'C': 10000.0, 'gamma': 0.01, 'kernel': 'rbf'}

***********************

Los hiperparámetros del mejor modelo son:
{'C': 10000.0, 'gamma': 0.01, 'kernel': 'rbf'}

****************************************************************************************************
P_IM_STEN
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en grilla en entrenamiento: 0.997
Exactitud luego de búsqueda aleatoria en grilla en validación: 0.835
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.9006410256410257
F1_score luego de búsqueda aleatoria en validación: 0.8424642139975314
****************************************************************************************************
647.206701040268


### 2.4. Naive Bayes

Entrenamos el modelo de Naive Bayes (Gausiano) de tipo clasificación.

In [10]:
from sklearn.naive_bayes import GaussianNB

In [11]:
modelNB = GaussianNB()
#modelNB.fit(X_train, y_train)

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

print('Los valores a probar en la búsqueda aleatoria son:')
pprint(params_NB)


for var_objetivo in var_objetivo_lst:
    # Lectura
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    X_test.drop(["var_objetivo"], axis=1, inplace=True)


    modelNB_random = RandomizedSearchCV(estimator = modelNB, 
                                   param_distributions=params_NB, 
                                   cv = 3,          # Validación cruzada 3-fold
                                   verbose=2, 
                                   random_state=0, 
                                   n_jobs = -1      # Paralelizar en todos los cores disponibles
                                   )
    modelNB_random.fit(X_train, y_train)

    ## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
    modelNB_best_model = modelNB_random.best_estimator_

    print('Los hiperparámetros del mejor modelo son:')
    pprint(modelNB_random.best_params_)
    print()


    
    
    
    y_pred = modelNB_best_model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    f1_score_metric = f1_score(y_test, y_pred, average='weighted')
    
    score_NB_train = modelNB_best_model.score(X_train, y_train)
    score_NB_test = modelNB_best_model.score(X_test, y_test)
    
    resultados_modelo_metricas_df=resultados_modelo_metricas_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'Modelo' : 'Naive Bayes - Gaussian' , 
            'Accuracy Train' : score_NB_train,
            'Accuracy Test' : score_NB_test,
            'Especificidad' : specificity,
            'F1_score': f1_score_metric
        },
        ignore_index=True
    )
    
    print('*'*100)
    print(f'{var_objetivo}')
    print('-'*100)

    print(f"Exactitud luego de búsqueda aleatoria en grilla en entrenamiento: {score_NB_train:.3f}")
    print(f"Exactitud luego de búsqueda aleatoria en grilla en validación: {score_NB_test:.3f}")
    print('.'*50)
    print('Especificidad luego de búsqueda aleatoria en validación:', specificity)
    print('F1_score luego de búsqueda aleatoria en validación:', f1_score_metric)
    print('*'*100)
    end = time.time()

Los valores a probar en la búsqueda aleatoria son:
{'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.84803587e-02, 2.31012970e-02, 1.87381742e-02,
       1.51991108e-02, 1.23284674e-02, 1.00000000e-02, 8.11130831e-03,
       6.57933225e-03, 5.33669923e-03, 4.32876128e-03, 3.51119173e-03,
       2.84803587e-03, 2.31012970e-03, 1.87381742e-03, 1.51991108e-03,
       1.23284674e-03, 1.00000000e-03, 8.11130831e-04, 6.57933225e-04,
       5.33669923e-04, 4.32876128e-04, 3.51119173e-04, 2.84803587e-04,
       2.31012970e-04, 1.87381742e-04, 1.51991108e-04, 1.23284674e-04,
       1.00000000e-04, 8.11130831e-05, 6.57933225e-05, 5.33669923e-05,
       4.32876128e-05, 3.51119173e-05, 2.84803587e-05, 2.31012970e-05,
       1

Los hiperparámetros del mejor modelo son:
{'var_smoothing': 0.004328761281083057}

****************************************************************************************************
ZSN
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en grilla en entrenamiento: 0.617
Exactitud luego de búsqueda aleatoria en grilla en validación: 0.418
..................................................
Especificidad luego de búsqueda aleatoria en validación: 0.3665338645418327
F1_score luego de búsqueda aleatoria en validación: 0.44343028050673516
****************************************************************************************************
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Los hiperparámetros del mejor modelo son:
{'var_smoothing': 0.03511191734215131}

****************************************************************************************************
REC_IM
-------------------

### 2.5. Redes Neuronales

Entrenamos el modelo de Redes Neuronales con Tensorflow/Keras.

In [20]:
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier

In [21]:
def create_model(
    activation='relu',
    capas_distr='7',
    x_shape=(380,1)
):
    """
    Se tomaron las arquitecturas descritas en diferentes articulos cientificos donde se trabajó con Redes Neuronales con el mismo 
    conjunto de datos. Los articulos científicos de referencia son:
        - Automatic Detection and Localization of Myocardial Infarction using Back Propagation Neural Networks 
          https://ieeexplore-ieee-org.ezproxybib.pucp.edu.pe/stamp/stamp.jsp?tp=&arnumber=5514664
        
        - Performance Analysis of Support Vector Machine and Neural Networks in Detection of Myocardial Infarction
          https://www.sciencedirect.com/science/article/pii/S1877050915000447
          
        - Prediction of incident myocardial infarction using machine learning applied to harmonized electronic health record data
          https://bmcmedinformdecismak.biomedcentral.com/track/pdf/10.1186/s12911-020-01268-x.pdf
    """
    
    # Mapeo de distribucion de neuronas por capa en Red Neuronal pasado por grilla
    if capas_distr == '7':
        neuronas_lst = [100,100,100,100,100,100,100]
    if capas_distr == '3-1':
        neuronas_lst = [50,40,30]
    elif capas_distr == '3-2':
        neuronas_lst = [30,40,50]
    elif capas_distr == '3-3':
        neuronas_lst = [16,10,4]
    elif capas_distr == '2-1':
        neuronas_lst = [30,15]
    elif capas_distr == '2-2':
        neuronas_lst = [30,40]
    elif capas_distr == '2-3':
        neuronas_lst = [50,60]
    elif capas_distr == '2-4':
        neuronas_lst = [70,80]
        
    # Crear arquitectura de capas de Red Reuronal según parámetro pasado por grilla
    if capas_distr == '7':
        model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=x_shape),
            tf.keras.layers.Dense(neuronas_lst[0], activation=activation),
            tf.keras.layers.Dense(neuronas_lst[1], activation=activation),
            tf.keras.layers.Dense(neuronas_lst[2], activation=activation),
            tf.keras.layers.Dense(neuronas_lst[3], activation=activation),
            tf.keras.layers.Dense(neuronas_lst[4], activation=activation),
            tf.keras.layers.Dense(neuronas_lst[5], activation=activation),
            tf.keras.layers.Dense(neuronas_lst[6], activation=tf.nn.softmax)
        ])
    elif ('3' in capas_distr):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=x_shape),
            tf.keras.layers.Dense(neuronas_lst[0], activation=activation),
            tf.keras.layers.Dense(neuronas_lst[1], activation=activation),
            tf.keras.layers.Dense(neuronas_lst[2], activation=tf.nn.softmax)
        ])
    elif ('2' in capas_distr):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=x_shape),
            tf.keras.layers.Dense(neuronas_lst[0], activation=activation),
            tf.keras.layers.Dense(neuronas_lst[1], activation=tf.nn.softmax)
        ])
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer= tf.optimizers.Adam(), metrics=['accuracy'])
    return model

In [22]:
for var_objetivo in var_objetivo_lst:
    # Lectura
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    X_test.drop(["var_objetivo"], axis=1, inplace=True)
    
    # Crear Modelo con Clasificador de Keras - Redes Neuronales en base a función genérica
    model = KerasClassifier(build_fn=create_model, epochs=100, validation_split=0.2, batch_size=600) ##batch_size=128 previous
    # Se define la grilla de búsqueda.
    # - Función de activación
    f_activacion = ['relu', 'tanh', 'sigmoid']
    # - Se contempla diferentes arquitecturas de redes neuronales tomados de la literatura
    capas_distr = ['7','3-1','3-2','3-3','2-1','2-2','2-3','2-4']
    # - Se contempla la dimensión de entrada del X
    x_shape = [X_train.iloc[0].shape]

    param_grid = dict(activation=f_activacion,capas_distr=capas_distr,x_shape=x_shape)


    random_grid = RandomizedSearchCV(estimator=model, 
                                     param_distributions = param_grid, 
                                     cv = 5,          # Validación cruzada 10-fold
                                     verbose=2, 
                                     random_state=2022, 
                                     n_iter = 20,
                                     n_jobs = -1)
    modelRN_random = random_grid.fit(X_train, y_train)

    # Resumen de Resultados
    print("Mejor: %f usando %s" % (random_grid.best_score_, random_grid.best_params_))
    means = modelRN_random.cv_results_['mean_test_score']
    stds = modelRN_random.cv_results_['std_test_score']
    params = modelRN_random.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    ## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
    modelRN_best_model = modelRN_random.best_estimator_
    
    train_acc = modelRN_best_model.score(X_train, y_train)
    test_acc = modelRN_best_model.score(X_test, y_test)
    
    y_pred = modelRN_best_model.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    f1_score_metric = f1_score(y_test, y_pred, average='weighted')

    resultados_modelo_metricas_df=resultados_modelo_metricas_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'Modelo' : 'Redes Neuronales' , 
            'Accuracy Train' : train_acc,
            'Accuracy Test' : test_acc,
            'Especificidad' : specificity,
            'F1_score': f1_score_metric
        },
        ignore_index=True
    )
    
    print('*'*100)
    print(f'{var_objetivo}')
    print('-'*100)
    print(f"Exactitud luego de búsqueda aleatoria en grilla en entrenamiento: {train_acc:.3f}")
    print(f"Exactitud luego de búsqueda aleatoria en grilla en validación: {test_acc:.3f}")
    print('.'*50)
    print('Especificidad luego de búsqueda aleatoria aleatoria en validación:', specificity)
    print('F1_score luego de búsqueda aleatoria en validación:', f1_score_metric)
    print('*'*100)

<ipython-input-22-39f15515b7a8>:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=100, validation_split=0.2, batch_size=600) ##batch_size=128 previous


Fitting 5 folds for each of 20 candidates, totalling 100 fits


D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Users\coliverac\Anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 236, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "D:\Users\coliverac\Anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit
    self.model = self.build_fn(

Epoch 1/100
4/4 [==============================] - 1s 75ms/step - loss: 4.5345 - accuracy: 0.2486 - val_loss: 4.3521 - val_accuracy: 0.2998
Epoch 2/100
4/4 [==============================] - 0s 15ms/step - loss: 4.2055 - accuracy: 0.6450 - val_loss: 3.8215 - val_accuracy: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 15ms/step - loss: 3.3774 - accuracy: 0.6413 - val_loss: 2.6235 - val_accuracy: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 15ms/step - loss: 1.7248 - accuracy: 0.6413 - val_loss: 1.8306 - val_accuracy: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 15ms/step - loss: 0.7984 - accuracy: 0.6413 - val_loss: 0.9121 - val_accuracy: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 15ms/step - loss: 0.7041 - accuracy: 0.4630 - val_loss: 1.1051 - val_accuracy: 0.0000e+00
Epoch 7/100
4/4 [==============================] - 0s 16ms/step - loss: 0.6701 - accuracy: 0.6413 - val_loss: 0.7927 - val_accuracy: 0.1530


Epoch 59/100
4/4 [==============================] - 0s 14ms/step - loss: 0.0109 - accuracy: 0.9979 - val_loss: 0.0184 - val_accuracy: 0.9958
Epoch 60/100
4/4 [==============================] - 0s 14ms/step - loss: 0.0147 - accuracy: 0.9953 - val_loss: 0.0466 - val_accuracy: 0.9874
Epoch 61/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0104 - accuracy: 0.9995 - val_loss: 0.0157 - val_accuracy: 0.9958
Epoch 62/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0063 - accuracy: 0.9995 - val_loss: 0.0184 - val_accuracy: 0.9979
Epoch 63/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.0187 - val_accuracy: 0.9979
Epoch 64/100
4/4 [==============================] - 0s 13ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0135 - val_accuracy: 0.9979
Epoch 65/100
4/4 [==============================] - 0s 14ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0157 - val_accuracy: 0.9958
Epoch 66/100


...

Epoch 1/100
4/4 [==============================] - 1s 64ms/step - loss: 4.6005 - accuracy: 0.0388 - val_loss: 4.5150 - val_accuracy: 0.0084
Epoch 2/100
4/4 [==============================] - 0s 13ms/step - loss: 4.4368 - accuracy: 0.6221 - val_loss: 4.2599 - val_accuracy: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 14ms/step - loss: 4.0295 - accuracy: 0.6415 - val_loss: 3.6067 - val_accuracy: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 13ms/step - loss: 3.0129 - accuracy: 0.6415 - val_loss: 2.2297 - val_accuracy: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 13ms/step - loss: 1.2758 - accuracy: 0.6415 - val_loss: 1.6862 - val_accuracy: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 14ms/step - loss: 0.7063 - accuracy: 0.6415 - val_loss: 0.5615 - val_accuracy: 1.0000
Epoch 7/100
4/4 [==============================] - 0s 14ms/step - loss: 0.6998 - accuracy: 0.4727 - val_loss: 1.5931 - val_accuracy: 0.0000e+00


Epoch 59/100
4/4 [==============================] - 0s 16ms/step - loss: 0.0302 - accuracy: 0.9921 - val_loss: 0.0653 - val_accuracy: 0.9937
Epoch 60/100
4/4 [==============================] - 0s 16ms/step - loss: 0.0327 - accuracy: 0.9906 - val_loss: 0.0229 - val_accuracy: 1.0000
Epoch 61/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0274 - accuracy: 0.9927 - val_loss: 0.0064 - val_accuracy: 1.0000
Epoch 62/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0313 - accuracy: 0.9911 - val_loss: 0.0397 - val_accuracy: 1.0000
Epoch 63/100
4/4 [==============================] - 0s 16ms/step - loss: 0.0211 - accuracy: 0.9969 - val_loss: 0.0079 - val_accuracy: 1.0000
Epoch 64/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0225 - accuracy: 0.9948 - val_loss: 0.0096 - val_accuracy: 1.0000
Epoch 65/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0226 - accuracy: 0.9942 - val_loss: 0.0348 - val_accuracy: 0.9979
Epoch 66/100


<ipython-input-22-39f15515b7a8>:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=100, validation_split=0.2, batch_size=600) ##batch_size=128 previous
D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_sco

Epoch 1/100
4/4 [==============================] - 1s 99ms/step - loss: 4.5471 - accuracy: 0.3023 - val_loss: 4.4533 - val_accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 18ms/step - loss: 4.2221 - accuracy: 0.6413 - val_loss: 4.0070 - val_accuracy: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 17ms/step - loss: 3.3628 - accuracy: 0.6413 - val_loss: 2.9277 - val_accuracy: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 19ms/step - loss: 1.6398 - accuracy: 0.6413 - val_loss: 2.5779 - val_accuracy: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 18ms/step - loss: 0.9729 - accuracy: 0.6413 - val_loss: 1.6041 - val_accuracy: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 17ms/step - loss: 0.7004 - accuracy: 0.5435 - val_loss: 0.4270 - val_accuracy: 1.0000
Epoch 7/100
4/4 [==============================] - 0s 17ms/step - loss: 0.7633 - accuracy: 0.4306 - val_loss: 1.2400 - val_accuracy: 0.0000e

Epoch 59/100
4/4 [==============================] - 0s 20ms/step - loss: 0.0391 - accuracy: 0.9912 - val_loss: 0.0462 - val_accuracy: 0.9959
Epoch 60/100
4/4 [==============================] - 0s 20ms/step - loss: 0.0369 - accuracy: 0.9928 - val_loss: 0.0230 - val_accuracy: 1.0000
Epoch 61/100
4/4 [==============================] - 0s 18ms/step - loss: 0.0357 - accuracy: 0.9896 - val_loss: 0.0347 - val_accuracy: 0.9979
Epoch 62/100
4/4 [==============================] - 0s 17ms/step - loss: 0.0319 - accuracy: 0.9938 - val_loss: 0.0392 - val_accuracy: 0.9938
Epoch 63/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0308 - accuracy: 0.9948 - val_loss: 0.0188 - val_accuracy: 1.0000
Epoch 64/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0303 - accuracy: 0.9938 - val_loss: 0.0484 - val_accuracy: 0.9917
Epoch 65/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0289 - accuracy: 0.9959 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 66/100


In [23]:
resultados_modelo_metricas_df

,Variable Objetivo,Modelo,Accuracy Train,Accuracy Test,Especificidad,F1_score
0,FIBR_PREDS,RandomForest,1.000000,0.897059,0.983713,0.867688
1,PREDS_TAH,RandomForest,0.998474,0.985294,1.000000,0.977996
2,JELUD_TAH,RandomForest,0.998839,0.979412,1.000000,0.969225
3,FIBR_JELUD,RandomForest,0.997253,0.944118,0.996894,0.919836
4,A_V_BLOK,RandomForest,1.000000,0.970588,1.000000,0.958553
...,...,...,...,...,...,...
61,RAZRIV,Redes Neuronales,1.000000,0.952941,0.978852,0.950071
62,DRESSLER,Redes Neuronales,1.000000,0.908824,0.953704,0.907423
63,ZSN,Redes Neuronales,0.997565,0.670588,0.820717,0.654276
64,REC_IM,Redes Neuronales,1.000000,0.864706,0.917722,0.871574


In [46]:
resultados_df = pd.DataFrame(columns=["Variable Objetivo","RandomForest","XGBoost","SVM","Naive Bayes(Gaussian)","Redes Neuronales","Métrica"])

In [47]:
for var_objetivo in resultados_modelo_metricas_df['Variable Objetivo'].unique().tolist():

    
    metrics_df = resultados_modelo_metricas_df.loc[
                    (resultados_modelo_metricas_df['Variable Objetivo'] == var_objetivo)
                ]
    
    resultados_df=resultados_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'RandomForest' : f'{metrics_df.loc[metrics_df["Modelo"]=="RandomForest","Accuracy Train"].iloc[0]*100 :.3f}%',
            'XGBoost' : f'{metrics_df.loc[metrics_df["Modelo"]=="XGBoost","Accuracy Train"].iloc[0]*100 :.3f}%',
            'SVM' : f'{metrics_df.loc[metrics_df["Modelo"]=="SVM","Accuracy Test"].iloc[0]*100 :.3f}%',
            'Naive Bayes(Gaussian)': f'{metrics_df.loc[metrics_df["Modelo"]=="Naive Bayes - Gaussian","Accuracy Train"].iloc[0]*100 :.3f}%',
            'Redes Neuronales' : f'{metrics_df.loc[metrics_df["Modelo"]=="Redes Neuronales","Accuracy Train"].iloc[0]*100 :.3f}%',
            'Métrica': "Accuracy Train"
        },
        ignore_index=True
    )
    
    resultados_df=resultados_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'RandomForest' : f'{metrics_df.loc[metrics_df["Modelo"]=="RandomForest","Accuracy Test"].iloc[0]*100 :.3f}%',
            'XGBoost' : f'{metrics_df.loc[metrics_df["Modelo"]=="XGBoost","Accuracy Test"].iloc[0]*100 :.3f}%',
            'SVM' : f'{metrics_df.loc[metrics_df["Modelo"]=="SVM","Accuracy Test"].iloc[0]*100 :.3f}%',
            'Naive Bayes(Gaussian)': f'{metrics_df.loc[metrics_df["Modelo"]=="Naive Bayes - Gaussian","Accuracy Test"].iloc[0]*100 :.3f}%',
            'Redes Neuronales' : f'{metrics_df.loc[metrics_df["Modelo"]=="Redes Neuronales","Accuracy Test"].iloc[0]*100 :.3f}%',
            'Métrica': "Accuracy Test"
        },
        ignore_index=True
    )
    
    resultados_df=resultados_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'RandomForest' : f'{metrics_df.loc[metrics_df["Modelo"]=="RandomForest","Especificidad"].iloc[0]*100 :.3f}%',
            'XGBoost' : f'{metrics_df.loc[metrics_df["Modelo"]=="XGBoost","Especificidad"].iloc[0]*100 :.3f}%',
            'SVM' : f'{metrics_df.loc[metrics_df["Modelo"]=="SVM","Especificidad"].iloc[0]*100 :.3f}%',
            'Naive Bayes(Gaussian)': f'{metrics_df.loc[metrics_df["Modelo"]=="Naive Bayes - Gaussian","Especificidad"].iloc[0]*100 :.3f}%',
            'Redes Neuronales' : f'{metrics_df.loc[metrics_df["Modelo"]=="Redes Neuronales","Especificidad"].iloc[0]*100 :.3f}%',
            'Métrica': "Especificidad"
        },
        ignore_index=True
    )
    
    resultados_df=resultados_df.append(
        {
            'Variable Objetivo' : var_objetivo,
            'RandomForest' : f'{metrics_df.loc[metrics_df["Modelo"]=="RandomForest","F1_score"].iloc[0]*100 :.3f}%',
            'XGBoost' : f'{metrics_df.loc[metrics_df["Modelo"]=="XGBoost","F1_score"].iloc[0]*100 :.3f}%',
            'SVM' : f'{metrics_df.loc[metrics_df["Modelo"]=="SVM","F1_score"].iloc[0]*100 :.3f}%',
            'Naive Bayes(Gaussian)': f'{metrics_df.loc[metrics_df["Modelo"]=="Naive Bayes - Gaussian","F1_score"].iloc[0]*100 :.3f}%',
            'Redes Neuronales' : f'{metrics_df.loc[metrics_df["Modelo"]=="Redes Neuronales","F1_score"].iloc[0]*100 :.3f}%',
            'Métrica': "F1_score"
        },
        ignore_index=True
    )

In [49]:
resultados_df.loc[resultados_df['Métrica']=="Accuracy Test"]

,Variable Objetivo,RandomForest,XGBoost,SVM,Naive Bayes(Gaussian),Redes Neuronales,Métrica
1,FIBR_PREDS,89.706%,87.941%,81.176%,60.588%,86.176%,Accuracy Test
5,PREDS_TAH,98.529%,98.529%,97.353%,42.941%,97.941%,Accuracy Test
9,JELUD_TAH,97.941%,96.765%,95.588%,77.059%,96.471%,Accuracy Test
13,FIBR_JELUD,94.412%,94.118%,92.059%,71.765%,93.235%,Accuracy Test
17,A_V_BLOK,97.059%,97.059%,92.941%,54.412%,96.176%,Accuracy Test
21,OTEK_LANC,90.882%,88.824%,86.471%,51.471%,90.000%,Accuracy Test
25,RAZRIV,97.059%,96.471%,95.294%,75.000%,95.588%,Accuracy Test
29,DRESSLER,95.294%,95.294%,88.235%,55.000%,91.765%,Accuracy Test
33,ZSN,78.529%,78.235%,65.294%,41.765%,62.059%,Accuracy Test
37,REC_IM,92.353%,91.176%,81.176%,42.059%,86.765%,Accuracy Test
